In [13]:
%cd tailscale_all

/home/griffin/git/griffinht.com/notebooks/tailscale_all


In [31]:
%%writefile pod.yaml
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: tailscale-state
spec:
  accessModes:
    # read/write from a single node
    # https://kubernetes.io/docs/concepts/storage/persistent-volumes/#access-modes
    - ReadWriteOnce
  # todo idk where the docs are for this - i just guessed
  resources:
    requests:
      storage: 10Mi
---
apiVersion: v1
kind: Pod
metadata:
    name: caddy
spec:
    containers:
      - name: iperf3
        image: networkstatic/iperf3
        args:
          - --server
      - name: caddy
        image: caddy/caddy
      - name: tailscale
        image: ghcr.io/tailscale/tailscale:latest
        env:
          # disable Kubernetes secrets and use OAuth login instead
          - name: TS_KUBE_SECRET
            value: ""
          - name: TS_STATE_DIR
            value: "/var/lib/tailscale"
        volumeMounts:
          - name: tailscale-state
            mountPath: "/var/lib/tailscale"
    volumes:
      - name: tailscale-state
        persistentVolumeClaim:
          claimName: tailscale-state


Overwriting pod.yaml


In [34]:
%%sh
kubectl apply -f pod.yaml

persistentvolumeclaim/tailscale-state created
pod/caddy created


Tip: If your pod is taking a while to come up, check out `kubectl describe`

In [24]:
%%sh
kubectl describe pod caddy

Name:             caddy
Namespace:        default
Priority:         0
Service Account:  default
Node:             gk3-my-cluster-pool-3-31f1207c-nfrt/10.142.0.7
Start Time:       Sun, 23 Jun 2024 18:25:21 -0400
Labels:           <none>
Annotations:      autopilot.gke.io/resource-adjustment:
                    {"input":{"containers":[{"name":"iperf3"},{"name":"caddy"},{"name":"tailscale"}]},"output":{"containers":[{"limits":{"cpu":"500m","ephemer...
                  autopilot.gke.io/warden-version: 2.9.37
Status:           Running
SeccompProfile:   RuntimeDefault
IP:               10.15.0.133
IPs:
  IP:  10.15.0.133
Containers:
  iperf3:
    Container ID:   containerd://b2bc732be4b3a938eea50dae2bb9e5dbdf8e14099c6fcf5d9a8c94dcaa501dd6
    Image:          networkstatic/iperf3
    Image ID:       docker.io/networkstatic/iperf3@sha256:148ec0389185caa366802a996ccded1bcde221a257f3eb44f291a7ac765f4ed4
    Port:           <none>
    Host Port:      <none>
    State:          Waiting
      Rea

In [28]:
%%sh
iperf3 -c caddy

Connecting to host caddy, port 5201
[  5] local 100.99.123.66 port 39614 connected to 100.123.205.111 port 5201
[ ID] Interval           Transfer     Bitrate         Retr  Cwnd
[  5]   0.00-1.00   sec  11.8 MBytes  98.5 Mbits/sec  1608    186 KBytes       
[  5]   1.00-2.00   sec  8.12 MBytes  68.2 Mbits/sec    0    211 KBytes       
[  5]   2.00-3.00   sec  8.25 MBytes  69.2 Mbits/sec    0    239 KBytes       
[  5]   3.00-4.00   sec  9.62 MBytes  80.7 Mbits/sec    5    199 KBytes       
[  5]   4.00-5.00   sec  6.88 MBytes  57.7 Mbits/sec   11    159 KBytes       
[  5]   5.00-6.00   sec  8.25 MBytes  69.2 Mbits/sec    0    187 KBytes       
[  5]   6.00-7.00   sec  8.12 MBytes  68.2 Mbits/sec    0    216 KBytes       
[  5]   7.00-8.00   sec  9.62 MBytes  80.7 Mbits/sec   25    174 KBytes       
[  5]   8.00-9.00   sec  8.12 MBytes  68.2 Mbits/sec    0    204 KBytes       
[  5]   9.00-10.00  sec  8.25 MBytes  69.2 Mbits/sec    0    227 KBytes       
- - - - - - - - - - - - - - - - 

It works! We added `iperf3` to our pod, and connected to it via Tailscale.

I want to separate the two.

In [10]:
%%writefile pod.yaml
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: tailscale-state
spec:
  accessModes:
    # read/write from a single node
    # https://kubernetes.io/docs/concepts/storage/persistent-volumes/#access-modes
    - ReadWriteOnce
  # todo idk where the docs are for this - i just guessed
  resources:
    requests:
      storage: 10Mi
---
apiVersion: v1
kind: Pod
metadata:
    name: iperf3
spec:
    containers:
      - name: iperf3
        image: networkstatic/iperf3
        args:
          - --server
---
apiVersion: v1
kind: Pod
metadata:
    name: caddy
spec:
    containers:
      - name: caddy
        image: caddy/caddy
---
apiVersion: v1
kind: Pod
metadata:
    name: tailscale
spec:
    containers:
      - name: tailscale
        image: ghcr.io/tailscale/tailscale:latest
        env:
          # disable Kubernetes secrets and use OAuth login instead
          - name: TS_KUBE_SECRET
            value: ""
          - name: TS_STATE_DIR
            value: "/var/lib/tailscale"
        volumeMounts:
          - name: tailscale-state
            mountPath: "/var/lib/tailscale"
    volumes:
      - name: tailscale-state
        persistentVolumeClaim:
          claimName: tailscale-state


Overwriting pod.yaml


In [11]:
! kubectl apply -f pod.yaml

persistentvolumeclaim/tailscale-state unchanged
pod/iperf3 unchanged
pod/caddy unchanged
pod/tailscale configured


In [8]:
! tailscale status | grep tailscale

100.113.140.69  tailscale            griffinht@   linux   -


Great! Now Tailscale shows up in its own pod as the "tailscale" hostname. We could also change the hostname if we wanted to, but I'll leave it like this for now.

In [9]:
! curl tailscale:80

curl: (7) Failed to connect to tailscale port 80 after 95 ms: Couldn't connect to server


In [ ]:
todo https://github.com/asciinema/discussions/issues/40

In [16]:

from IPython.display import HTML
HTML("""<script src="https://asciinema.org/a/1wkyfecyj2qa4t9gm4bank61y.js" id="asciicast-1wkyfecyj2qa4t9gm4bank61y" async></script>""")

In [19]:
%%html
<script src="https://asciinema.org/a/1wkyfecyj2qa4t9gm4bank61y.js" id="asciicast-1wkyfecyj2qa4t9gm4bank61y" async></script>

In [14]:
! asciinema play tailscale.cast

◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡◠◡
0C 07:51:21 m(B9s(Bs(B
=(B                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                (B[38:5:214;40m ____  __.________      _________ .____    .___ (B                                                                                    

In [23]:
%%writefile service.yaml
apiVersion: v1
kind: Service
metadata:
    name: caddy-service
spec:
    selector:
        app: caddy
    ports:
      - protocol: TCP
        port: 80
---
apiVersion: v1
kind: Service
metadata:
    name: iperf-service
spec:
    selector:
        app: iperf3
    ports:
      - protocol: TCP
        port: 5201
        # todo does iperf also need UDP?
---


Overwriting service.yaml


Think this looks complex? Think again! Check out my lab on container-container networking in Docker Compose. I haven't created it yet but I want to!

This medium article on [Mastering Kubernetes Pod-to-Pod Communication](https://medium.com/@extio/mastering-kubernetes-pod-to-pod-communication-a-comprehensive-guide-46832b30556b) was also great.

In [24]:
! kubectl apply -f service.yaml

service/caddy-service created
service/iperf-service created


In [26]:
! kubectl exec tailscale -- wget caddy-service:80

Connecting to caddy-service:80 (10.96.43.142:80)
wget: can't connect to remote host (10.96.43.142): Connection refused
command terminated with exit code 1
